In [17]:
import boto3
import random
#import dotenv
#load env from .env
#dotenv.load_dotenv()
bedrock_runtime = boto3.client('bedrock-runtime')

In [18]:
import tiktoken
from anthropic_bedrock import AnthropicBedrock
anthropic_bedrock = AnthropicBedrock(
    aws_region='us-east-1',
)



def num_tokens_from_string(text, model_name):
    if model_name.startswith("gpt"):
        # encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
        pass
        # num_tokens = len(encoding.encode(text))
    elif model_name.startswith("claude"):
        num_tokens = anthropic_bedrock.count_tokens(text)  # 
    return num_tokens

In [19]:

from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage,AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [20]:

llm_sonnet = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                  model_kwargs={"temperature": 0.2,
                                "top_k":250,
                                "max_tokens": 1024,
                                "top_p":0.95,
                                # "stop_sequences":['</response>']
                               },
                  streaming=False,callbacks=[StreamingStdOutCallbackHandler()])

In [21]:
template_extract = \
"""
You are helpful assistant who is an expert in natural language processing and especially Named entity recognition.
Create a JSON response that categorizes words from the provided text into specific keys.
The keys should include People, Country, Location,  City, Manufacturer, Brand, Datetime, Event, and Other.
please enclose the answer in xml tag <output>

The values of these keys should be lists containing relevant words from the input text.
Example:
input text: 我想了解海尔在美国有关新品发布的新闻报道
<output>
{{
"brand":"海尔",
"country":"美国",
"event":"新品发布",
}}
</output>

Here is user's input text:
<query>
{query}
</query>

Skip the preamble, go straight into the answer.

"""

In [22]:
template_code = \
"""
You are developer of mastering Elasticsearch, now you are developing application on Elasticsearch,
Here is the schema of the Elasticsearch you can refer to:
<schema>
{schema}
</schema>

Here is the user's query and keywords extracted from the query you can use:
<query>
{query}
</query>

<keywords>
{keywords}
</keywords>

please translate user's query to Elasticsearch DSL (Domain Specific Language) for query the data in Elasticsearch engine.
please enclose the answer in xml tag <output>
Skip the preamble, go straight into the answer.

"""

In [34]:
schema = \
    """
    字段名 | 数据类型 | 字段说明 |
---|--- | ---
gather.site_name | keyword | 网站名称 :'新浪网'
gather.site_domain | keyword | 网站主域名 ：'sina.com.cn'
gather.site_country | keyword | 网站所在国家 :'中国'
gather.site_location | keyword | 网站归属地 ：'北京'
gather.site_lang_code | keyword | 网站使用语言代码,  采用ISO639-1标准
gather.ICP | keyword | 网站ICP： '粤B2-20090059-5'
gather.gtime | timestamp | 采集时间，时间戳
gather.config_id | keyword | 配置id
gather.data_type | keyword | 数据来源类型
gather._data_type | keyword | 兼容秘书data_type字段
gather.site_level.weight  | byte | 网站分级,取值范围[0,9]:
gather.site_level.name | keyword | 网站分级名称
gather.info_flag | array<keyword> |  '01'：新闻  '02'：论坛  '03'：博客  '04'：微博 '0401':新浪微博 '0408'：新浪长微博 '0105'：平媒 '06'：微信 '07'： 视频 '0109'：APP '11'： 小视频 '12'：境外资讯 '1201': 境外 '1202'：外文 '13': 境外社交媒体 '1301': Twitter '1302': Facebook  '99':搜索 '17':电视监控 info_flag是array类型,如微博info_flag:[04, 0401]
uuid | keyword | 文章uuid
url  | keyword | 文章格式化后的链接
title   | text | 标题
content   | text | 内容
ctime   | timestamp | 发布时间
channel | keyword | 频道
breadcrump | array<keyword> | 面包屑:  ['天涯论坛', '经济论坛']
layout_num | keyword | 信息为平煤时，所在版面
article_keywords | array<keyword> | 文章标签、关键词
meta_keywords | text | 对应html meta标签中keywords属性
repost_source | keyword | 文章来源、转载来源
repost_count | integer | 转发数
reply_count | integer | 评论数
visit_count | integer | 阅读数、播放数
like_count | integer | 点赞数
share_count | integer | 分享数
bullet_count | integer | 弹幕数
coins_count | integer | 硬币数
zaikan_count | integer | 在看数 (微信在看数据)
appear_time | date | 时间戳，弹幕弹出时间
collection_count | integer | 收藏数
edit_count | integer |发生编辑次数、内容编辑次数
is_junk | integer | 是否是垃圾网站、无价值数据
duration | integer | 视频长度。单位为秒
music_id | keyword | 音乐id。短视频中背景音乐id
pid | keyword | 转发、转载、评论等信息的 上一级信息id
rootid | keyword |评论信息中，用于标记多级评论信息的根信息id。 例如：一条微博有多条评论，其中一条评论(id=011)又被多人评论， 这些子评论的rootid就是011
wtype | byte | 发表类型:    1: 原创   2: 转发  7: 评论  8:弹幕   9: 转载  21: 原创保护<微信>
place | keyword | 签到地点
surface_img |  |封面图片url
star_surface_img |  |封面图片下载后，保存在星光服务器上的url;
device | keyword | 发帖设备
is_toutiao | byte | 是否是头条（注：主要是微信）  0:否  1:头条、置顶 2:精华帖 4:满级精华 5:推荐 6:首页推荐 7:作者认证  支持array类型：[2, 6]表示同时是精华帖和首页推荐
is_purchased | byte | 是否付费  1:需要
deleted | byte | 是否以删除： 0: 正常访问  1:已删除 2:设置访问权限
pic_urls |  | 图片链接,可以是多个   ['http://weibo.com/1.jpg',   'http://tieba.baidu.com/1.jpg']
video_urls | | 视频链接,可以是多个 ['http://weibo.com/a/b.mp4',]
has_attributes | array<keyword> | 内容标注：   has_pic:包含图片  has_videos:包含视频  has_slink:包含短链   
translation | array<object> | 翻译结果  包含 title, content, lang_code, title_server, content_server字段;    [{  'content':'这个建筑',  'title':'河流',  'lang_code':'zh_cn',  'title_server':2,   'content_server':1  }]
translation.lang_code | keyword | 翻译语言代码,默认为zh-cn
translation.title | text | 标题翻译后结果
translation.content | text | 内容翻译后结果
translation.title_server | byte | 翻译方式： 1. 百度；  2. 谷歌；  3. 民族翻译据；  4. 自建翻译；  5. 新译
translation.content_server | byte | 翻译方式
captions | array<object> | 字幕或者语音识结果  [{   'detail': [   {   "start": 12.3,     "dur": 8.7,     "text": 'hello'   ] }   "language_code": "en"  }]
captions.detail.start | integer | 开始时间，取相对时间
captions.detail.dur | integer | 时间跨度
captions.detail.text | text | 字幕内容、语音识别结果
captions.language_code | keyword | 语言编码
data_subscribe.customer_id_list | array[string] | 对上的道丁客户id(全局唯一)
data_subscribe.d_id_list | array[string] | 对上的道丁账号id(全局唯一)
data_subscribe.user_id_list | array[string] | 对上的数据平台用户id
data_subscribe.user_subject_id_list | array[string] | 对上的数据平台用户专题id(全局唯一)
slink_info | array<object> | 短链信息
slink_info.display_name | keyword | 显示名称
slink_info.summary | keyword | 短链备注,描述信息
slink_info.ct | timestamp | 短链接创建时间
slink_info.url_short | keyword | 短链接
slink_info.url_long | keyword | 原始链接
slink_info.object_type | keyword | 链接类型,如:   link : 链接    webpage : 网页    place : 签到地点    cardlist    video : 视频    collection
editors | array<keyword> | 编辑
author  | array<keyword> | 作者. 保存新闻(非自媒体)、微信中的作者信息 |
publisher.name | keyword | 发布者名称 |
publisher.id | keyword | 发布者id |
publisher.platform | keyword | 发布平台   自媒体、新闻、平媒 |
publisher.entity | keyword | 发布主体 |
publisher.site_name | keyword | 发布网站名称 |
user.name | keyword | 用户名 |
user.nickname | keyword | 用户昵称
user.uid | keyword | 用户uid
user.url | keyword | 用户个人主页地址
user.gender | keyword | 用户性别： 'm':男  'f':女 'n':未知
user.profile_img_url | keyword | 用户头像url
user.friends_count | integer | 用户关注数
user.followers_count | integer | 用户粉丝数
user.bi_followers_count | integer | 用户的互粉数
user.statuses_count | integer | 用户发文数
user.visit_count | integer | 用户作品总访问数、播放量
user.like_count | integer | 用户作品总获赞数、喜欢数
user.created_at | integer | 用户注册时间
user.location | array<keyword> | 用户地域，采集所得数据 ['上海', '静安区']
user.country | keyword | 用户所在国家,标准化后的数据
user.province | keyword | 用户所在省份,标准化后的数据
user.city | keyword | 用户所在区县,标准化后的数据
user.level | integer | 用户级别
user.verified | byte | 用户是否认证: 0:未认证 1:普通认证(对应微博橙v) 2:机构认证(对应微博蓝v)
user.verified_type | short | 认证类型 -1:普通用户 0:名人 1:政府 2:企业 3：媒体 4:校园 5:网站 6:应用 7:团体(机构) 8:待审企业 10:类政府 200:初级达人 220:中高级达人 400:已故V用户
user.verified_reason | object | 认证描述、认证原因
user.description | text | 用户描述、个人简介
user.ip_region|array<string>|ip属地（注：暂支持的有 微博、微信、头条、知乎）
user.kwaiId | string | 快手号
analysis.sentiment | integer | 正负面： -6:确定负面(高敏感) -5:确定负面(敏感) -2:确定负面 -1:疑似负面 0:中性 1:疑似正面 2:确定正面 9：争议 
analysis.noise | integer | 噪音  0: 非噪音  1:噪音
analysis.keywords | array<object> | 分析所得关键词  [ {'word':‘电视剧’},    {'word':‘偶像’}  ]
analysis.hashcode.1 | float | 雷同数据排重，用于排除雷同数据;ti+cn+siteName=>hash(排重)
analysis.hashcode.2 | float | hash2:近似数据排重，用于聚合几乎相同数据;title 5个关键词计算HASH
analysis.hashcode.3 | float | hash3:相同主题排重，用户聚合相同主题数据; kw5 5个关键词计算HASH
analysis.hashcode.4 | float | hash4:事件聚合指纹，用户聚合同一事件数据; sim_hash2(事件分类hash值)
analysis.hashcode.5 | float | hash5:宽泛主题分类，用户聚合类似主题数据; kw5结果排序去掉后面2个词，再计算一次hash值
analysis.tag | array<object> | 标签字段 [{'cid':‘12’, 'tid':‘23’, ‘words’:['北京', '海淀']}]
analysis.tag.cid | keyword | 标签对应客户id
analysis.tag.tid | keyword | 标签id
analysis.tag.words | keyword | 匹配词
analysis.classification | array<object> | 系统自动分类[{'name':'金融'，'weight':80}]
analysis.classification.name | keyword | 自动分类结果名称: 如汽车、金融
analysis.classification.weight | byte | 自动分类权重[0, 100]，取整数
retweeted | object | 转发信息中，原发(被转)文章信息； 评论数据中，主贴或原文信息； 字段定义同原文信息，参见上面字段定义
analysis.find_address.province | array | 省份列表(从信息中通过地域识别获取): ['河南省', '湖北省']
analysis.find_address.city | array | 城市列表(从信息中通过地域识别获取): ['武汉市', '郑州市']0
analysis.find_address.district | array | 地区列表(从信息中通过地域识别获取): ['吉利区', '金水区']
analysis.find_address.province_count | integer | 地域识别：省份个数
analysis.find_address.city_count | integer | 地域识别：城市个数
analysis.find_address.district_count | integer | 地域识别：区县个数
analysis.info_src.cls.name | array<keyword> | 信源分类名称
analysis.info_src.cls.level | keyword | 信源分类级别
analysis.info_src.loc.name | array<keyword> | 信源所属地域
analysis.info_src.loc.level | keyword | 信源所属地域级别
analysis.info_src.lv.name | keyword | 信源分级名称
analysis.info_src.lv.level | keyword | 信源分级级别
analysis.info_src.type | keyword | 匹配规则类型；1：域名匹配  2：自媒体账号匹配
analysis.info_src.domain | keyword | 匹配规则时对应的domain信息
analysis.info_src.name | keyword | 网站名、板块名、频道名
analysis.hashtag | array | 微博话题
analysis.mentions | array | 微博中被 @ 的用户
    """

In [24]:
prompt_extract = ChatPromptTemplate.from_template(template_extract)
prompt_code = ChatPromptTemplate.from_template(template_code)

In [25]:
from langchain_core.output_parsers.base import BaseOutputParser
import re
from operator import itemgetter
class CustOuputParser(BaseOutputParser[str]):

    def extract(self,content:str) -> str:
        pattern = r"<output>(.*?)</output>"
        match = re.search(pattern, content, re.DOTALL)
        return match.group(1) if match else 'Empty'      
    
    def parse(self, text: str) -> str:
        cleaned_text = self.extract(text)
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"

In [26]:
output_parser = CustOuputParser()

llm = llm_sonnet
chain_1 = prompt_extract | llm |output_parser
chain_2 = prompt_code | llm|output_parser

In [27]:
chain_3 = ({'keywords':chain_1,'query':itemgetter('query'),"schema":itemgetter('schema')})|prompt_code|llm|output_parser

In [28]:
query = "我想了解最近一个月美国媒体关于海尔的负面新闻报道"

In [29]:
chain_1.invoke({"query":query,"schema":schema})

'\n{\n"Brand": ["海尔"],\n"Country": ["美国"],\n"Datetime": ["最近一个月"],\n"Event": ["负面新闻报道"]\n}\n'

In [30]:
resp = chain_3.invoke({"query":query,"schema":schema})

In [31]:
print(resp)


{
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "gather.site_country": "美国"
          }
        },
        {
          "match_phrase": {
            "content": "海尔"
          }
        },
        {
          "range": {
            "ctime": {
              "gte": "now-1M/d",
              "lte": "now/d"
            }
          }
        },
        {
          "range": {
            "analysis.sentiment": {
              "lte": -1
            }
          }
        }
      ]
    }
  }
}



In [33]:
query2 = "我想了解近期海尔有哪些营销活动"
resp2 = chain_3.invoke({"query":query2,"schema":schema})
print(resp2)


{
  "query": {
    "bool": {
      "must": [
        {
          "match_phrase": {
            "content": "海尔"
          }
        },
        {
          "match_phrase": {
            "content": "营销活动"
          }
        },
        {
          "range": {
            "ctime": {
              "gte": "now-90d/d",
              "lte": "now/d"
            }
          }
        }
      ]
    }
  },
  "sort": [
    {
      "ctime": {
        "order": "desc"
      }
    }
  ]
}

